# P1 - Edwin Wray

- All queries must use the ANSI 92 standard for queries with the type safe “on” and will have relational output
    
- Output should be in a JSON component.
    

Queries should be in one of two formats

1. Medium query should have from 2 to 3 tables joined and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables.
    
2. Complex query should have from 3 or more tables joined, a custom scalar function, and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables.

* * *

## Medium

Q1

Proposition: Write a query that returns the number of Employees, Customers and Total Sales in  each country.  

Table: Sales.Customer,  Sales.\[Order\] and  SALES.OrderDetail  tables  

Column: C.CustomerCountry, E.EmployeeId,  C.CustomerId,  OD.UnitPrice  and OD.Quantity  

Predicate: There are no predicates in this query.

In [ ]:
-- Q1 
USE Northwinds2022TSQLV7;

-- Returns the number of Employees, Customers and Total Sales in  each country.
SELECT 
    C.CustomerCountry AS 'Country'
    , (
        -- Correlated Subquery that returns the count of employees in the current country
        SELECT COUNT(E.EmployeeId) AS 'NumberOfEmployees'
        FROM HumanResources.Employee AS E
        WHERE E.EmployeeCountry = C.CustomerCountry
    ) AS 'NumberOfEmployees'
    , COUNT(C.CustomerId) AS 'NumberOfCustomers'
    , CONCAT('$', SUM(OD.UnitPrice * OD.Quantity)) AS 'TotalSales'
FROM 
    Sales.Customer AS C
    LEFT JOIN Sales.[Order] AS O 
        ON O.CustomerId = C.CustomerId
    LEFT JOIN SALES.OrderDetail AS OD 
        ON OD.OrderId = O.OrderId
GROUP BY 
    C.CustomerCountry
ORDER BY 
    C.CustomerCountry

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q2

Proposition: Write a query that returns the each shipping companies first orderdate, last orderdate, number of orders and average number of days before shipping  

Table: Sales.\[Order\]  and  Sales.Shipper tables.  

Column: S.ShipperId, S.ShipperCompanyName,  FirstOrderDate,  LastOrderDate,  TotalOrders and AvgNumDaysBeforeShipping.  

Predicate: There are no predicates in this query.

In [ ]:
-- Q2 
USE Northwinds2022TSQLV7;

SELECT 
    S.ShipperId, S.ShipperCompanyName
    , MIN(O.OrderDate) AS 'FirstOrderDate'
    , MAX(O.OrderDate) AS 'LastOrderDate'
    , COUNT(DISTINCT O.OrderId) AS 'TotalOrders'
    , (
        -- Correlated Subquery that returns the average number of days before shipping an order for each shipper
        SELECT AVG(DATEDIFF(DAY, O2.ShipToDate, O2.RequiredDate))
        FROM Sales.[Order] AS O2
        WHERE O2.ShipperId = S.ShipperId
    ) AS 'AvgNumDaysBeforeShipping'
FROM Sales.[Order] AS O 
    LEFT JOIN Sales.Shipper as S
        ON S.ShipperId = O.ShipperId
GROUP BY 
    S.ShipperId, S.ShipperCompanyName
ORDER BY 
    S.ShipperId

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q3

Proposition: Write a query that returns a table of suppliers and their contact info in countries with employees.

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Table:&nbsp;</span>  

Column: 

Predicate:

In [ ]:
-- Q3 
USE Northwinds2022TSQLV7;

-- CTE returning each supplier's contact information
;WITH SupplierContactInfoCTE
AS
(
    SELECT S.SupplierId, S.SupplierCompanyName
    , S.SupplierContactName, S.SupplierContactTitle
    , S.SupplierPhoneNumber, S.SupplierFaxNumber
    , S.SupplierCountry
    FROM Production.Supplier AS S 
)

SELECT 
    E.EmployeeCountry, COUNT(DISTINCT E.EmployeeId) AS 'NumberOfEmployees'
    , CTE.SupplierId, CTE.SupplierCompanyName
    , CTE.SupplierContactName, CTE.SupplierContactTitle
    , CTE.SupplierPhoneNumber, COALESCE(CTE.SupplierFaxNumber, 'No Fax Number') AS SupplierFaxNumber
FROM 
    SupplierContactInfoCTE AS CTE 
    LEFT JOIN Production.Product AS P 
        ON CTE.SupplierId = P.SupplierId
    LEFT JOIN Sales.OrderDetail AS OD 
        ON P.ProductId = OD.ProductId
    LEFT JOIN Sales.[Order] AS O 
        ON O.OrderId = OD.OrderId
    RIGHT JOIN HumanResources.Employee AS E 
        ON E.EmployeeId = O.EmployeeId
WHERE 
    E.EmployeeCountry = CTE.SupplierCountry
GROUP BY 
    E.EmployeeCountry
    , CTE.SupplierId, CTE.SupplierCompanyName
    , CTE.SupplierContactName, CTE.SupplierContactTitle
    , CTE.SupplierPhoneNumber, CTE.SupplierFaxNumber
ORDER BY 
    E.EmployeeCountry, CTE.SupplierId

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q4

Proposition: Write query that returns the last order placed by the 10 newest customers.  

Table: Sales.Customer and  Sales.\[Order\] table.  

Column: C.CustomerId, O.OrderId and O.OrderDate.  

Predicate: There are no predicates in this query.

In [ ]:
-- Q4 
USE Northwinds2022TSQLV7;

-- CTE that returns the last order of each customer
;WITH CustOrdersCTE
AS
(
    SELECT C.CustomerId, MAX(O.OrderId) AS 'OrderId', MAX(O.OrderDate) AS 'OrderDate'
    FROM Sales.Customer AS C 
        INNER JOIN Sales.[Order] AS O 
            ON C.CustomerId = O.CustomerId
    GROUP BY C.CustomerId
)

-- Query that returns the last order of the 10 newest customers
SELECT TOP (10) CTE.CustomerId, CTE.OrderId, CTE.OrderDate
FROM CustOrdersCTE AS CTE
ORDER BY CTE.CustomerId DESC

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q5

Proposition: Write a query that returns the last date each product was ordered and how many days have past since that order date.  

Table: Sales.OrderDetail  and  Sales.\[Order\] tables.  

Column: OD.ProductId,  O.OrderId and  O.OrderDate.  

Predicate:There are no predicates in this query.

In [ ]:
-- Q5 
USE Northwinds2022TSQLV7;

-- View returns each product's last order
GO
CREATE OR ALTER VIEW ProductLastOrderedVIEW
AS
SELECT OD.ProductId, MAX(O.OrderId) AS 'OrderId', MAX(O.OrderDate) AS 'LastOrderDate'
FROM Sales.OrderDetail AS OD 
    INNER JOIN Sales.[Order] O 
        ON OD.OrderId = O.OrderId
GROUP BY OD.ProductId
GO

-- Query returns the last date each product was ordered and how many days have past since that order date.  
SELECT 
    V.ProductId, V.OrderId, V.LastOrderDate
    , DATEDIFF(DAY, V.LastOrderDate, (SELECT MAX(O.OrderDate) FROM Sales.[Order] AS O)) AS 'DaysSinceLastOrderDate'
FROM 
    ProductLastOrderedVIEW AS V
ORDER BY 
    V.ProductId

- * * *
    

## Medium

Q6

Proposition: Display the yearly total number products sold and revenue generated.  

Table: Sales.SalesOrderDetail  AND  Sales.SalesOrderHeader.  

Column: YEAR(OH.OrderDate)  AS 'OrderYear',  OD.OrderQty, OD.UnitPrice,  SUM(OrderQty)  AS 'TotalProducts' and  SUM(Sales.FormatMoney(OrderQty, UnitPrice))  AS  'TotalRevenue'.  

Predicate: No Predicates

In [ ]:
-- Q6 
USE AdventureWorks2017;
GO

-- Scalar Function that formats the money
CREATE OR ALTER FUNCTION Sales.FormatMoney
(
    -- Define input parameters
    @OrderQty INT,
    @UnitPrice MONEY
)
RETURNS MONEY
AS
BEGIN
    -- Declare variable(s)
    DECLARE @TotalRevenue MONEY;
    SET @TotalRevenue = @OrderQty * @UnitPrice;

    RETURN @TotalRevenue;
END;

GO

-- CTE gathering order info
WITH CustomerOrderDate AS
(
    SELECT
        YEAR(OH.OrderDate) AS 'OrderYear'
        , OD.OrderQty
        , OD.UnitPrice
    FROM Sales.SalesOrderDetail AS OD
        LEFT JOIN Sales.SalesOrderHeader AS OH
        ON OD.SalesOrderID = OH.SalesOrderID
)
-- Displaying order info
SELECT
    OrderYear
    , SUM(OrderQty) AS 'TotalProducts'
    , CONCAT('$', ROUND(SUM(Sales.FormatMoney(OrderQty, UnitPrice)), 2)) AS 'TotalRevenue'
FROM CustomerOrderDate
GROUP BY OrderYear
ORDER BY OrderYear

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q7

Proposition: Write a query that returns the Total Price and Total Discounted Price of orders in December.  

Table: Sales.OrderDetail  AS OD,  Sales.\[Order\]  AS O and  Sales.Customer  AS C.  

Column: OD.OrderID, OD.ProductID, OD.UnitPrice, OD.Quantity and  OD.DiscountPercentage.  

Predicate: MONTH(O.OrderDate) = 12   

> Returns true if the orderdate month is December.

In [ ]:
-- Q7 
USE Northwinds2022TSQLV7;

-- Define a scalar function to calculate total discounted price
GO
CREATE OR ALTER FUNCTION Sales.DiscountedTotalPrice(@UnitPrice MONEY, @Quantity INT, @DiscountPercentage FLOAT)
RETURNS MONEY
AS
BEGIN
    DECLARE @totalPrice MONEY;
    SET @totalPrice = @unitPrice * @quantity * (1 - @DiscountPercentage);
    RETURN @totalPrice;
END
GO

-- CTE to get order details
;WITH OrderDetailsCTE 
AS 
(
    SELECT 
        OD.OrderID
        , OD.ProductID
        , OD.UnitPrice
        , OD.Quantity
        , OD.DiscountPercentage
    FROM 
        Sales.OrderDetail AS OD
)

--  Query that returns detailed info information about orders placed in December
SELECT
    O.OrderDate
    , O.OrderID
    , C.CustomerId
    , SUM(CTE.UnitPrice * Quantity) AS 'TotalPrice'
    , SUM(Sales.DiscountedTotalPrice(CTE.UnitPrice, CTE.Quantity, CTE.DiscountPercentage)) AS 'DiscountedTotalPrice'
FROM 
    Sales.[Order] AS O 
    INNER JOIN Sales.Customer AS C 
         ON C.CustomerID = O.CustomerID
    INNER JOIN OrderDetailsCTE AS CTE 
        ON CTE.OrderID = O.OrderID
WHERE
   MONTH(O.OrderDate) = 12
GROUP BY
    O.OrderDate
    , O.OrderId
    , C.CustomerId
ORDER BY
    O.OrderDate
    , O.OrderId
    , C.CustomerId

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q8

Proposition: Display the cars in stock, the cost of a given model and cost of the model's total inventory.  

Table: Data.Make  AS Make,  Data.Model  AS Model,  Output.StockPrices  AS StockPrices and  Data.Stock  AS Stock.  

Column: StockPrices.MakeName, StockPrices.ModelName,  COUNT(Stock.ModelID)  AS 'InventoryCount',  CONCAT('$', Stock.Cost)  AS 'Cost' and CONCAT('$',  SUM(Stock.Cost))  AS 'InventoryCost'  

Predicate: No Predicates.

In [ ]:
-- Q8 
USE PrestigeCars
GO

-- CTE returning a car's make name and model id
WITH MakeModel(MakeName, ModelID) AS 
(
    SELECT Make.MakeName, Model.ModelID
    FROM Data.Make AS Make 
        INNER JOIN Data.Model AS Model
        ON Make.MakeID = Model.MakeID
)

-- Displaying cost of a model and that model's total inventory
SELECT 
    StockPrices.MakeName
    , StockPrices.ModelName
    , COUNT(Stock.ModelID) AS 'InventoryCount'
    , CONCAT('$', Stock.Cost) AS 'Cost'
    , CONCAT('$', SUM(Stock.Cost)) AS 'InventoryCost'
FROM 
    MakeModel AS Make 
    LEFT JOIN Output.StockPrices AS StockPrices
        ON Make.MakeName = StockPrices.MakeName
    INNER JOIN Data.Stock AS Stock
        ON Make.ModelID = Stock.ModelID
GROUP BY 
    StockPrices.MakeName
    , StockPrices.ModelName
    , Stock.Cost
ORDER BY 
    Stock.Cost DESC
    , StockPrices.MakeName
    , StockPrices.ModelName
    , InventoryCount

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q9

Proposition: A query that returns the top 100 express delivery orders in Q3 2014 (2 or less days) and its delivery instructions.  

Table: Sales.\[Orders\]  AS O,  Sales.Invoices  AS I and  Sales.InvoiceLines  AS I2.  

Column: O2.OrderDate, O2.ExpectedDeliveryDate, O2.OrderID and CONCAT('$',  SUM(I2.UnitPrice  \* Quantity))  AS 'TotalPrice'  

Predicate: O2.OrderDate  \>= '20140701' AND  O2.OrderDate  \< '20141001'. This predicate in the Where clause filters the table to only include orders dated July 2014 through September 2014 inclusively.

In [ ]:
-- Q9 
USE WideWorldImporters
GO

-- Return all express delivery orders in Q3 2014 (2 or less days)
SELECT 
    O2.OrderDate
    , O2.ExpectedDeliveryDate
    , O2.OrderID
    , CONCAT('$', SUM(I2.UnitPrice * Quantity)) AS 'TotalPrice'
FROM 
( -- Inner Query return all customers with express delivery (2 or less days)
    SELECT
        O.OrderDate, O.ExpectedDeliveryDate, O.OrderID
    FROM
        Sales.[Orders] AS O
    WHERE 
        DATEDIFF(day, O.OrderDate, O.ExpectedDeliveryDate) <= 2
) AS O2
    LEFT JOIN Sales.Invoices AS I
        ON O2.OrderID = I.OrderID
    LEFT JOIN Sales.InvoiceLines AS I2
        ON I2.InvoiceID = I.InvoiceID

-- Filter table for order in Q3 2014: July-Sept 2014
WHERE 
    O2.OrderDate >= '20140701' AND O2.OrderDate < '20141001'
GROUP BY 
    O2.OrderDate
    , O2.ExpectedDeliveryDate
    , O2.OrderID
ORDER BY 
    O2.OrderDate
    , O2.ExpectedDeliveryDate
    , O2.OrderID

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q10

Proposition: Write a query that returns   <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">&nbsp;customers with 5 or more orders in a given year.</span>

Table: <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.[Order] </span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">AS</span> <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">O and </span> <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.Customer </span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">as</span> <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">C.</span>

Column: <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">O2.ShipToYear</span><span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">, O2.CustomerId</span><span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">, O2.CustomerContactName</span><span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">, </span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(121, 94, 38);">COUNT</span><span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">(O2.OrderId) </span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">AS</span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(163, 21, 21);">'NumberOrders'</span>

Predicate:

In [ ]:
-- Q10 
USE Northwinds2022TSQLV7
GO

-- Outer Query displaying only customers with >= 5 orders in a given year
SELECT 
    O2.ShipToYear
    , O2.CustomerId
    , O2.CustomerContactName
    , COUNT(O2.OrderId) AS 'NumberOrders' 
FROM
(
    -- Inner Query that returns all of a customer's orders
    SELECT 
        YEAR(O.ShipToDate) as ShipToYear
        , C.CustomerId
        , O.OrderId
        , C.CustomerContactName
    FROM Sales.[Order] AS O
        LEFT JOIN Sales.Customer as C
            ON O.CustomerID = C.CustomerId
) AS O2

GROUP BY 
    O2.ShipToYear
    , O2.CustomerId
    , O2.CustomerContactName
HAVING 
    COUNT(O2.OrderId) > 4
ORDER BY 
    O2.ShipToYear
    , O2.CustomerId

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q11  Proposition: A query that returns the number of days to remanufacture products sold and its restock date.  

Table: Production.Product  AS P and  Sales.SalesOrderDetail  AS OD  

Column: SalesOrderDetailID,  ProductID,  DaysToManufacture,  CEILING(P2.DaysToManufacture  \* @BufferDays)  as RestockDate,  ROW\_NUMBER()  OVER (ORDER BY P2.SalesOrderDetailID)  AS RowNum  

Predicate: No Predicate.

In [ ]:
-- Q11
USE AdventureWorks2017
GO

DECLARE @BufferDays FLOAT = 1.5;
DECLARE @PageSize INT = 20; -- Define the number of rows per page
DECLARE @PageNumber INT = 1; -- Define the page number you want to retrieve

-- CTE that returns the number of days to remanufacture products sold
WITH DaysManProd AS 
(
    SELECT
        OD.SalesOrderDetailID
        , OD.ProductID
        , P.DaysToManufacture
    FROM 
        Production.Product AS P
        INNER JOIN Sales.SalesOrderDetail AS OD
            ON P.ProductID = OD.ProductID
)

-- Add ROW_NUMBER() to the CTE
, PageCTE AS 
(
    SELECT
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
        , CEILING(P2.DaysToManufacture * @BufferDays) as RestockDate
        , ROW_NUMBER() OVER (ORDER BY P2.SalesOrderDetailID) AS RowNum
    FROM
        DaysManProd as P2
    GROUP BY
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
)

-- Use OFFSET and FETCH to segment the results
SELECT
    SalesOrderDetailID
    , ProductID
    , DaysToManufacture
    , RestockDate
FROM
    PageCTE
WHERE
    RowNum BETWEEN ((@PageNumber - 1) * @PageSize + 1) AND (@PageNumber * @PageSize);

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q12

Proposition: Write a query that returns the total revenue made each year in the first week of each month.  

Table: Data.Sales  AS S and  Data.Customer  AS C  

Column: YEAR(SaleDate)  AS 'SaleDateYear' and CONCAT('$',  SUM(SC.TotalSalePrice))  AS 'FirstWeek OfEachMonth TotalSales'  

Predicate: DAY(S.SaleDate) \< 8 in the WHERE clause filters the table to only orders dated in the first week of each month.

In [ ]:
-- Q12 
USE PrestigeCars;
GO

-- VIEW that returns the totalsaleprice of each order 
-- dated in the first week of each month
CREATE OR ALTER VIEW Data.SalesCustomer
AS
SELECT
    S.SaleDate
    , S.SalesID
    , S.TotalSalePrice
FROM 
    Data.Sales AS S
    INNER JOIN Data.Customer AS C
        ON S.CustomerID = C.CustomerID
WHERE
    DAY(S.SaleDate) < 8
GO

-- Query that returns the total revenue made each year in the first week of each month.
SELECT 
    YEAR(SaleDate) AS 'SaleDateYear'
    , CONCAT('$', SUM(SC.TotalSalePrice)) AS 'FirstWeek OfEachMonth TotalSales'
FROM  
    Data.SalesCustomer AS SC
GROUP BY
    YEAR(SaleDate)
ORDER BY 
    YEAR(SaleDate)

-- FOR JSON AUTO, include_null_values;

* * *

## Medium

Q13

Proposition: Query that returns the lowest cost of each make and model that's in stock. 

Table: Data.Make AS FindMake, Data.Model AS FindModel and   Data.Stock  AS S.  

Column: FindModel.ModelID, FindMake.MakeName,  FindModel.ModelName, and CONCAT('$', MIN(S.Cost)) AS Cost  

Predicate: No Predicate.

In [ ]:
-- Q13 
USE PrestigeCars
GO

-- VIEW that returns all cars in the inventory by make, model and country
CREATE OR ALTER VIEW Data.FindCar
AS
SELECT
    FindMake.MakeName
    , FindModel.ModelName
    , FindModel.ModelID
FROM
    Data.Make AS FindMake
    INNER JOIN Data.Model AS FindModel
        ON FindMake.MakeID = FindModel.MakeID
GO

-- Query to returns the lowest cost of each make and model that's in stock
SELECT
    S.ModelID
    , F.MakeName
    , F.ModelName
    , CONCAT('$', MIN(S.Cost)) AS Cost
FROM 
    Data.FindCar AS F
    LEFT JOIN Data.Stock AS S
        ON F.ModelID = S.ModelID
GROUP BY
    S.ModelID
    , F.MakeName
    , F.ModelName
ORDER BY 
    F.MakeName
    , F.ModelName

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q14

Proposition: Write a query that returns the employee of the month for each month, determined by their total sales.

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Table: HumanResources.Employee,&nbsp; Sales.[Order]&nbsp; and&nbsp; Sales.OrderDetail tables.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Column: O.OrderDate,&nbsp; E.EmployeeId,&nbsp; E.EmployeeFirstName,&nbsp; E.EmployeeLastName,&nbsp; OD.UnitPrice&nbsp; and&nbsp; OD.Quantity.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Predicate: The predicates in the CASE statement evaluates the numerical representation of a month and assingns its respective alphabetical name.</span>

SET @MonthAlpha =

         CASE @MonthInt

             WHEN 1 THEN 'January'

             ...           ...            ...

             WHEN 12 THEN 'December'

         END

In [ ]:
-- Q14 
USE Northwinds2022TSQLV7;

-- Scalar Function that returns the alphabetical name of a numerical month
GO
CREATE OR ALTER FUNCTION Sales.MonthNumToAlphaFunc (@MonthInt INT)
RETURNS NVARCHAR(10)
AS
BEGIN
    DECLARE @MonthAlpha NVARCHAR(10);
    -- CASE that conditionally updates the numerical month to alphabetical
    SET @MonthAlpha =
        CASE @MonthInt
            WHEN 1 THEN 'January'
            WHEN 2 THEN 'February'
            WHEN 3 THEN 'March'
            WHEN 4 THEN 'April'
            WHEN 5 THEN 'May'
            WHEN 6 THEN 'June'
            WHEN 7 THEN 'July'
            WHEN 8 THEN 'August'
            WHEN 9 THEN 'September'
            WHEN 10 THEN 'October'
            WHEN 11 THEN 'November'
            WHEN 12 THEN 'December'
        END
    RETURN @MonthAlpha;
END;
GO

-- CTE that returns the total monthly sales of all employees
;WITH EmployeeTotalSalesCTE
AS
(
    SELECT 
        YEAR(O.OrderDate) AS 'OrderDateYear'
        , MONTH(O.OrderDate) AS 'OrderDateMonth'
        , E.EmployeeId
        , CONCAT(E.EmployeeFirstName, ' ', E.EmployeeLastName) AS 'EmployeeFullName'
        , CONCAT('$', SUM(OD.UnitPrice * OD.Quantity)) AS 'TotalSales'
    FROM 
        HumanResources.Employee AS E 
        LEFT JOIN Sales.[Order] AS O 
            ON O.EmployeeId = E.EmployeeId
        LEFT JOIN Sales.OrderDetail AS OD 
            ON OD.OrderId = O.OrderId
    GROUP BY
        YEAR(O.OrderDate), MONTH(O.OrderDate)
        , E.EmployeeId, CONCAT(E.EmployeeFirstName, ' ', E.EmployeeLastName)
)

-- Outer Query that returns the employee with highest total sales of the month for each month
SELECT 
    CTE.OrderDateYear
    , Sales.MonthNumToAlphaFunc(CTE.OrderDateMonth) AS 'OrderDateMonth'
    , CTE.EmployeeId
    , CTE.EmployeeFullName AS 'EmployeeOfTheMonth'
    , CTE2.TotalSales
FROM 
    EmployeeTotalSalesCTE AS CTE
    INNER JOIN 
        -- Inner Query that returns the highest total sales for each month
        (
            SELECT 
                CTE2.OrderDateYear
                , CTE2.OrderDateMonth
                , MAX(CTE2.TotalSales) AS 'TotalSales'
            FROM 
                EmployeeTotalSalesCTE AS CTE2
            GROUP BY
                CTE2.OrderDateYear
                , CTE2.OrderDateMonth

        ) AS CTE2
        ON CTE2.OrderDateYear = CTE.OrderDateYear 
            AND CTE2.OrderDateMonth = CTE.OrderDateMonth 
            AND CTE2.TotalSales = CTE.TotalSales
ORDER BY 
        CTE.OrderDateYear, CTE.OrderDateMonth, CTE.EmployeeId

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q15

Proposition: Query returns Total Revenue from each customer whose CustomerId is divisible by 4 and calculate total tax paid.

Table:  Data\].Customer  AS  C,  \[Data\].Sales  AS  S and \[Data\].SalesDetails AS SD  

Column: C.CustomerID, C.CustomerName, S.SalesID, SD.SalesDetailsID and SD.SalePrice  

Predicate: X.CustomerID %  4 = 0. This predicate in the WHERE clause filters the table to only return customers with id's divisible by 4.

In [ ]:
-- Q15 
USE PrestigeCars
GO

-- Scalar Function calculating total tax
CREATE OR ALTER FUNCTION Reference.TotalRevenueTax
(
    -- PARAMETERS
    @Amount DECIMAL(20, 2)
)
RETURNS DECIMAL(20, 2) AS
BEGIN
    --  Body
    DECLARE @Tax DECIMAL(20, 2)
    SELECT @Tax = @Amount * 0.15
    RETURN @Tax
END
GO

-- Outer Query returns Total Revenue from each customer whose CustomerId is divisible by 4 and calculate total tax paid.
SELECT 
    X.CustomerID
    , X.CustomerName
    , CONCAT('$', SUM(CAST(X.SalePrice AS DECIMAL(20, 2)))) AS 'TotatlRevenue'
    , CONCAT('$', Reference.TotalRevenueTax(SUM(CAST(X.SalePrice AS DECIMAL(20, 2))))) AS 'TotalRevenueTax'
FROM
(
    -- Inner Query gathering customers purchases
    SELECT 
        C.CustomerID
        , C.CustomerName
        , S.SalesID
        , SD.SalesDetailsID
        , SD.SalePrice
    FROM 
        [Data].Customer AS C 
        INNER JOIN [Data].Sales AS S 
            ON C.CustomerID = S.CustomerID
        INNER  JOIN [Data].SalesDetails AS SD
            ON SD.SalesID = S.SalesID

) AS X
WHERE
    X.CustomerID % 4 = 0
GROUP BY 
    X.CustomerID
    , X.CustomerName
ORDER BY 
    X.CustomerID

-- FOR JSON AUTO, include_null_values;


* * *

## Complex

Q16

Proposition: Create an email address for each employee, lastName.FirstName@Department.com. Display each employee's ID, name, email, and monthly number of orders. Order by the month.  

Table: HumanResources.Employee  AS Emp,  Triggered.Employee  AS EmpTrig and  EmployeeMonthlyOrders  as EmpMonth  

Column: E.EmployeeId,  YEAR(O.OrderDate)  AS  OrderYear,  MONTH(O.OrderDate)  AS  OrderMonth,  COUNT(DISTINCT  O.OrderId)  AS  NumOrders and HumanResources.EmployeeEmail(Emp.EmployeeLastName, Emp.EmployeeFirstName).  

Predicate: No Predicate.

In [ ]:
-- Q16 
USE Northwinds2022TSQLV7
GO

-- Scalar Function creating employe email addresses 
-- (lastName.FirstName@Department.com)
CREATE OR ALTER FUNCTION HumanResources.EmployeeEmail
(
    -- Parameters
    @LastName NVARCHAR(25),
    @FirstName NVARCHAR(25),
    @Department NVARCHAR(50)
)
RETURNS NVARCHAR(125) AS
BEGIN
    --  Body
    DECLARE @result NVARCHAR(125)
    SELECT @result = CONCAT(@LastName, '.', @FirstName, '@', 
                @Department, '.com')
    RETURN @result
END
GO

-- CTE returning employee's number of monthly orders
WITH EmployeeMonthlyOrders AS 
(
    SELECT
        E.EmployeeId
        , YEAR(O.OrderDate) AS OrderYear
        , MONTH(O.OrderDate) AS OrderMonth
        , COUNT(DISTINCT O.OrderId) AS NumOrders
    FROM 
        HumanResources.Employee as E
        LEFT JOIN Sales.[Order] as O 
            ON E.EmployeeId = O.EmployeeId
    GROUP BY
        E.EmployeeId
        , YEAR(O.OrderDate)
        , MONTH(O.OrderDate)
)

-- Displaying employees' email, info and number of monthly orders
SELECT 
        empMonth.OrderYear, empMonth.OrderMonth, Emp.EmployeeId, Emp.EmployeeLastName, Emp.EmployeeFirstName,
        HumanResources.EmployeeEmail(Emp.EmployeeLastName, Emp.EmployeeFirstName, EmpTrig.Department) as EmployeeEmail,
        empMonth.NumOrders
FROM 
    HumanResources.Employee AS Emp
    INNER JOIN Triggered.Employee AS EmpTrig
        ON Emp.EmployeeId = EmpTrig.EmployeeId
    LEFT JOIN EmployeeMonthlyOrders as EmpMonth
        ON Emp.EmployeeId = EmpMonth.EmployeeId
ORDER BY 
    EmpMonth.OrderYear
    , empMonth.OrderMonth
    , Emp.EmployeeId

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q17

Proposition: A query that returns a sales person's id, contact info, sales last year, quota of the Employee with the highest performance in respect to their quota (sales last year - quota).  

Table: Person.Person  as P,  Sales.SalesPerson  AS S and  Person.PersonPhone  as  P  

Column: S.BusinessEntityID, S.LastName, S.FirstName, P.PhoneNumber, S.SalesQuota, S.SalesLastYear and ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2) AS 'SalesOutperformance'  

Predicate: ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2) =

               (                 SELECT MAX(ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2)) as SalesReview

                 FROM Sales.SalesPerson AS S

              ) 

This predicate in the where clause filters for the sales person with the highest outperformance in respect to their quota.

In [ ]:
-- Q17 
USE AdventureWorks2017
GO

-- Scalar Function that calculates the difference btwn
-- a SalesPerson's sales last year and quota
CREATE OR ALTER FUNCTION Sales.SalesReview
(
    -- PARAMETERS
    @LastYearSales MONEY,
    @Quota MONEY
)
RETURNS MONEY AS
BEGIN
    --  Body
    DECLARE 
        @Result MONEY
    SELECT @Result = @LastYearSales - @Quota
    RETURN @Result
END
GO


-- VIEW that returns a SalesPerson's id, name, quota and last year sales
CREATE OR ALTER VIEW Sales.SalesPerformance
AS
SELECT
    P.BusinessEntityID
    , P.LastName, 
    P.FirstName
    , S.SalesQuota
    , S.SalesLastYear
FROM
    Person.Person as P
    INNER JOIN Sales.SalesPerson AS S
        ON P.BusinessEntityID = S.BusinessEntityID
GO

-- Returns a SalesPerson's with the highest outperformance. Returns their id, contact info, 
-- sales last year, quota, and performance review (sales last year - quota)
SELECT
    S.BusinessEntityID
    , S.LastName
    , S.FirstName
    , P.PhoneNumber
    , S.SalesQuota
    , S.SalesLastYear
    , ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2) AS 'SalesOutperformance'
FROM
    Sales.SalesPerformance as S
    INNER JOIN Person.PersonPhone as P 
        ON S.BusinessEntityID = P.BusinessEntityID
WHERE
    S.SalesQuota IS NOT NULL
    AND ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2) = 
            (
                SELECT MAX(ROUND(Sales.SalesReview(S.SalesLastYear, S.SalesQuota), 2)) as SalesReview
                FROM Sales.SalesPerson AS S
            )
GROUP BY 
    S.BusinessEntityID, S.LastName, S.FirstName, 
    P.PhoneNumber, S.SalesQuota, S.SalesLastYear
ORDER BY
    S.BusinessEntityID, S.LastName, S.FirstName

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q18

Proposition: A query that returns employees in a requested country. Return the employees' id, name, country, currency, year hired and age when hired.  

Table: dbo.DimEmployee  as e,  dbo.DimSalesTerritory as s and   dbo.DimCurrency  as c.  

Column: e.EmployeeKey, e.lastName, e.FirstName,  e.SalesTerritoryCountry, c.CurrencyName,  YEAR(e.HireDate)  AS  HireYear and  dbo.AgeWhenHired(e.BirthDate, e.HireDate)  AS  AgeWhenHired  

Predicate: No Predicate.

In [ ]:
-- Q18 
USE AdventureWorksDW2017
GO

-- Scalar Function returns Employee's age when they were hired
CREATE OR ALTER FUNCTION dbo.AgeWhenHired
(
    -- PARAMETERS
    @dateOfBirth DATE,
    @hireDate DATE
)
RETURNS INT AS
BEGIN
    --  Body
    DECLARE @result INT
    SELECT @result = DATEDIFF(YEAR, @dateOfBirth, @hireDate)
    RETURN @result
END
GO

-- Variable to input a sales country
DECLARE @salesCountry NVARCHAR(50) = 'CANADA'

-- Outer Query that returns info for employees in a requested country:
-- id, name, country, currency, year hired, and age when hired
SELECT DISTINCT
    e.EmployeeKey, e.lastName, e.FirstName,
    e.SalesTerritoryCountry, c.CurrencyName,
    YEAR(e.HireDate) AS HireYear,
    dbo.AgeWhenHired(e.BirthDate, e.HireDate) AS AgeWhenHired
FROM
(
    -- Inner Query that returns info for employees in a requested country
    SELECT 
        e.SalesTerritoryKey, e.EmployeeKey, e.lastName, e.FirstName,
        s.SalesTerritoryCountry, e.BirthDate, e.HireDate
    FROM 
        dbo.DimEmployee as e
        LEFT JOIN dbo.DimSalesTerritory as s
            ON e.SalesTerritoryKey = s.SalesTerritoryKey
    WHERE
        s.SalesTerritoryCountry = @salesCountry
) AS e LEFT JOIN dbo.FactInternetSales as f
        ON e.SalesTerritoryKey = f.SalesTerritoryKey
    LEFT JOIN dbo.DimCurrency as c
        ON f.CurrencyKey = c.CurrencyKey
GROUP BY
    e.EmployeeKey, e.lastName, e.FirstName,
    e.SalesTerritoryCountry, c.CurrencyName,
    e.HireDate, e.BirthDate
ORDER BY
    e.EmployeeKey, c.CurrencyName

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q19

Proposition: A query that returns a credit card's info and all orders placed after it expired.  

Table: Sales.SalesOrderHeader  AS o,  Sales.CreditCard  AS card and  Sales.Customer  AS cust  

Column: o.CustomerID, cust.AccountNumber, card.CardType,  Sales.CardLastFourDigits(card.CardNumber)  AS  CardLastFourDigits,  DATEFROMPARTS(card.ExpYear, card.ExpMonth,  1)  AS  ExpirationDate and  CAST(o.OrderDate  AS DATE)  AS  OrderDate  

Predicate: No Predicate.

In [ ]:
-- Q19
USE AdventureWorks2017
GO

-- Scalar Function that returns the last 4 digits of a credit card number
CREATE OR ALTER FUNCTION Sales.CardLastFourDigits
(
    -- PARAMETER
    @CreditCardNumber NVARCHAR(25)
)
RETURNS NVARCHAR(25) AS
BEGIN
    --  Body
    DECLARE @LastFourChar NVARCHAR(4) = Right(@CreditCardNumber, 4)

    RETURN @LastFourChar
END
GO

-- Outer Query that returns all orders placed after a 
-- credit card's info and all orders placed after it expired
SELECT DISTINCT
    o.CustomerID, cust.AccountNumber, card.CardType, 
    Sales.CardLastFourDigits(card.CardNumber) AS CardLastFourDigits,
    DATEFROMPARTS(card.ExpYear, card.ExpMonth, 1) AS ExpirationDate,
    CAST(o.OrderDate AS DATE) AS OrderDate
FROM 
    (
        -- Inner Query that returns a credit card's info and orders placed on it
        SELECT
            o.CustomerID, o.CreditCardID, o.SalesOrderID, o.OrderDate
        FROM 
            Sales.SalesOrderHeader AS o
        WHERE o.OrderDate IS NOT NULL
        GROUP BY o.CustomerID, o.CreditCardID, o.SalesOrderID, o.OrderDate
    ) AS o
    LEFT JOIN Sales.CreditCard AS card
        ON o.CreditCardID = card.CreditCardID
    LEFT JOIN Sales.Customer AS cust 
        ON o.CustomerID = cust.CustomerID
GROUP BY 
    o.CustomerID, cust.AccountNumber, card.CardType, 
    card.CardNumber, card.ExpYear, card.ExpMonth, o.OrderDate
HAVING 
    COUNT(DISTINCT o.SalesOrderID) > 0
ORDER BY
    o.CustomerID, OrderDate

-- FOR JSON AUTO, include_null_values;

* * *

## Complex

Q20

Proposition: Query that returns customers with an outstanding balance, displaying their total outstanding balance, collections status, and last order date.  

Table: Sales.CustTotOutstandingBalance  AS t,  Sales.\[Customers\]  AS c and  Sales.\[Orders\]  AS o  

Column: t.CustomerID, c.CustomerName, t.TotalOutstandingBalance,  Sales.CustomerCollections(t.TotalOutstandingBalance)  AS CollectionsAccount and  MAX(o.OrderDate)  AS LastOrderDate  

Predicate: No Predicate

In [ ]:
-- Q20
USE WideWorldImporters
GO

-- Scalar Function returns whether or not a customer's account is in collections
CREATE OR ALTER FUNCTION Sales.CustomerCollections
(
    -- PARAMETER
    @amount DECIMAL (18, 2)
)
RETURNS NVARCHAR(3) AS
BEGIN
    --  Body
    DECLARE @result NVARCHAR(3) = 
        CASE
            -- customer's account is in collections
            WHEN @amount >= 10000 THEN 'YES'
            -- customer's account is not in collections
            ELSE 'NO'
        END
    
    RETURN @result
END
GO

-- VIEW that returns each customer's total outstanding balance
CREATE OR ALTER VIEW Sales.CustTotOutstandingBalance
AS
SELECT
    t.CustomerID,
    SUM(t.OutstandingBalance) AS TotalOutstandingBalance
FROM Sales.CustomerTransactions AS t 
WHERE t.OutstandingBalance > 0
GROUP BY t.CustomerID
GO

-- Query that returns customers with an outstanding balance, displaying their 
-- total outstanding balance, collections status, and last order date
SELECT 
    t.CustomerID, c.CustomerName, t.TotalOutstandingBalance,
    Sales.CustomerCollections(t.TotalOutstandingBalance) AS CollectionsAccount,
    MAX(o.OrderDate) AS LastOrderDate
FROM 
    Sales.CustTotOutstandingBalance AS t 
    LEFT JOIN Sales.[Customers] AS c
        ON t.CustomerID = c.CustomerID
    LEFT JOIN Sales.[Orders] AS o
        ON t.CustomerID = o.CustomerID
GROUP BY
    t.CustomerID, c.CustomerName, t.TotalOutstandingBalance
ORDER BY
    t.CustomerID

-- FOR JSON AUTO, include_null_values;

# P1 - Edwin Wray

# Top Query

* * *

## Medium

Q2

Proposition: Write a query that returns each shipping company's first orderdate, last orderdate, number of orders and average number of days before shipping  

Table: Sales.\[Order\]  and  Sales.Shipper tables.  

Column: S.ShipperId, S.ShipperCompanyName,  FirstOrderDate,  LastOrderDate,  TotalOrders and AvgNumDaysBeforeShipping.  

Predicate: There are no predicates in this query.

In [ ]:
-- Q2 
USE Northwinds2022TSQLV7;

SELECT 
    S.ShipperId, S.ShipperCompanyName
    , MIN(O.OrderDate) AS 'FirstOrderDate'
    , MAX(O.OrderDate) AS 'LastOrderDate'
    , COUNT(DISTINCT O.OrderId) AS 'TotalOrders'
    , (
        -- Correlated Subquery that returns the average number of days before shipping an order for each shipper
        SELECT AVG(DATEDIFF(DAY, O2.ShipToDate, O2.RequiredDate))
        FROM Sales.[Order] AS O2
        WHERE O2.ShipperId = S.ShipperId
    ) AS 'AvgNumDaysBeforeShipping'
FROM Sales.[Order] AS O 
    LEFT JOIN Sales.Shipper as S
        ON S.ShipperId = O.ShipperId
GROUP BY 
    S.ShipperId, S.ShipperCompanyName
ORDER BY 
    S.ShipperId

-- FOR JSON AUTO, include_null_values;

# Top Query

* * *

## Medium

Q4

Proposition: Write query that returns the last order placed by the 10 newest customers.  

Table: Sales.Customer and  Sales.\[Order\] table.  

Column: C.CustomerId, O.OrderId and O.OrderDate.  

Predicate: There are no predicates in this query.

In [ ]:
-- Q4 
USE Northwinds2022TSQLV7;

-- CTE that returns the last order of each customer
;WITH CustOrdersCTE
AS
(
    SELECT 
        C.CustomerId
        , MAX(O.OrderId) AS 'OrderId'
        , MAX(O.OrderDate) AS 'OrderDate'
    FROM Sales.Customer AS C 
        INNER JOIN Sales.[Order] AS O 
            ON C.CustomerId = O.CustomerId
        INNER JOIN Sales.OrderDetail AS OD 
            ON OD.OrderId = O.OrderId
    GROUP BY C.CustomerId
)

-- Query that returns the last order of the 10 newest customers
SELECT TOP (10) CTE.CustomerId, CTE.OrderId, CTE.OrderDate
FROM CustOrdersCTE AS CTE
ORDER BY CTE.CustomerId DESC

-- FOR JSON AUTO, include_null_values;

# Top Query

* * *

## Complex

Q14

Proposition: Write a query that returns the employee of the month for each month, determined by their total sales.

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Table: HumanResources.Employee,&nbsp; Sales.[Order]&nbsp; and&nbsp; Sales.OrderDetail tables.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Column: O.OrderDate,&nbsp; E.EmployeeId,&nbsp; E.EmployeeFirstName,&nbsp; E.EmployeeLastName,&nbsp; OD.UnitPrice&nbsp; and&nbsp; OD.Quantity.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Predicate: The predicates in the CASE statement evaluates the numerical representation of a month and assingns its respective alphabetical name.</span>

SET @MonthAlpha =

         CASE @MonthInt

             WHEN 1 THEN 'January'

             ...           ...            ...

             WHEN 12 THEN 'December'

         END

In [ ]:
-- Q14 
USE Northwinds2022TSQLV7;

-- Scalar Function that returns the alphabetical name of a numerical month
GO
CREATE OR ALTER FUNCTION Sales.MonthNumToAlphaFunc (@MonthInt INT)
RETURNS NVARCHAR(10)
AS
BEGIN
    DECLARE @MonthAlpha NVARCHAR(10);
    -- CASE that conditionally updates the numerical month to alphabetical
    SET @MonthAlpha =
        CASE @MonthInt
            WHEN 1 THEN 'January'
            WHEN 2 THEN 'February'
            WHEN 3 THEN 'March'
            WHEN 4 THEN 'April'
            WHEN 5 THEN 'May'
            WHEN 6 THEN 'June'
            WHEN 7 THEN 'July'
            WHEN 8 THEN 'August'
            WHEN 9 THEN 'September'
            WHEN 10 THEN 'October'
            WHEN 11 THEN 'November'
            WHEN 12 THEN 'December'
        END
    RETURN @MonthAlpha;
END;
GO

-- CTE that returns the total monthly sales of all employees
;WITH EmployeeTotalSalesCTE
AS
(
    SELECT 
        YEAR(O.OrderDate) AS 'OrderDateYear'
        , MONTH(O.OrderDate) AS 'OrderDateMonth'
        , E.EmployeeId
        , CONCAT(E.EmployeeFirstName, ' ', E.EmployeeLastName) AS 'EmployeeFullName'
        , CONCAT('$', SUM(OD.UnitPrice * OD.Quantity)) AS 'TotalSales'
    FROM 
        HumanResources.Employee AS E 
        LEFT JOIN Sales.[Order] AS O 
            ON O.EmployeeId = E.EmployeeId
        LEFT JOIN Sales.OrderDetail AS OD 
            ON OD.OrderId = O.OrderId
    GROUP BY
        YEAR(O.OrderDate), MONTH(O.OrderDate)
        , E.EmployeeId, CONCAT(E.EmployeeFirstName, ' ', E.EmployeeLastName)
)

-- Outer Query that returns the employee with highest total sales of the month for each month
SELECT 
    CTE.OrderDateYear
    , Sales.MonthNumToAlphaFunc(CTE.OrderDateMonth) AS 'OrderDateMonth'
    , CTE.EmployeeId
    , CTE.EmployeeFullName AS 'EmployeeOfTheMonth'
    , CTE2.TotalSales
FROM 
    EmployeeTotalSalesCTE AS CTE
    INNER JOIN 
        -- Inner Query that returns the highest total sales for each month
        (
            SELECT 
                CTE2.OrderDateYear
                , CTE2.OrderDateMonth
                , MAX(CTE2.TotalSales) AS 'TotalSales'
            FROM 
                EmployeeTotalSalesCTE AS CTE2
            GROUP BY
                CTE2.OrderDateYear
                , CTE2.OrderDateMonth

        ) AS CTE2
        ON CTE2.OrderDateYear = CTE.OrderDateYear 
            AND CTE2.OrderDateMonth = CTE.OrderDateMonth 
            AND CTE2.TotalSales = CTE.TotalSales
ORDER BY 
        CTE.OrderDateYear, CTE.OrderDateMonth, CTE.EmployeeId

-- FOR JSON AUTO, include_null_values;

# Worst Query

* * *

## Medium

Q5

Proposition: Write a query that returns the last date each product was ordered and how many days have past since that order date.  

Table: Sales.OrderDetail  and  Sales.\[Order\] tables.  

Column: OD.ProductId,  O.OrderId and  O.OrderDate.  

Predicate:There are no predicates in this query.

In [ ]:
-- Q5 
USE Northwinds2022TSQLV7;

-- View returns each product's last order
GO
CREATE OR ALTER VIEW ProductLastOrderedVIEW
AS
SELECT OD.ProductId, MAX(O.OrderId) AS 'OrderId', MAX(O.OrderDate) AS 'LastOrderDate'
FROM Sales.OrderDetail AS OD 
    INNER JOIN Sales.[Order] O 
        ON OD.OrderId = O.OrderId
GROUP BY OD.ProductId
GO

-- Query returns the last date each product was ordered and how many days have past since that order date.  
SELECT 
    V.ProductId, V.OrderId, V.LastOrderDate
    , DATEDIFF(DAY, V.LastOrderDate, (SELECT MAX(O.OrderDate) FROM Sales.[Order] AS O)) AS 'DaysSinceLastOrderDate'
FROM 
    ProductLastOrderedVIEW AS V
ORDER BY 
    V.ProductId

-- FOR JSON AUTO, include_null_values;

# Corrected Query

## Medium

Q5

Proposition: Write a query that returns the last date each product was ordered and how many days have past since that order date.  

Corrected: Used SYSDATETIME() in Column DATEDIFF(DAY, V.LastOrderDate, SYSDATETIME()) AS 'DaysSinceLastOrderDate' so the result is dynamic.

In [ ]:
-- Q5 CORRECTED
USE Northwinds2022TSQLV7;

-- View returns each product's last order
GO
CREATE OR ALTER VIEW ProductLastOrderedVIEW
AS
SELECT OD.ProductId, MAX(O.OrderId) AS 'OrderId', MAX(O.OrderDate) AS 'LastOrderDate'
FROM Sales.OrderDetail AS OD 
    INNER JOIN Sales.[Order] O 
        ON OD.OrderId = O.OrderId
GROUP BY OD.ProductId
GO

-- Query returns the last date each product was ordered and how many days have past since that order date.  
SELECT V.ProductId, V.OrderId, V.LastOrderDate, DATEDIFF(DAY, V.LastOrderDate, SYSDATETIME()) AS 'DaysSinceLastOrderDate'
FROM ProductLastOrderedVIEW AS V
ORDER BY V.ProductId

-- FOR JSON AUTO, include_null_values;

# WORST QUERY

## Medium

Q11  

Proposition: A query that returns the number of days to remanufacture products sold and its restock date.  

Table: Production.Product  AS P and  Sales.SalesOrderDetail  AS OD  

Column: SalesOrderDetailID,  ProductID,  DaysToManufacture,  CEILING(P2.DaysToManufacture  \* @BufferDays)  as RestockDate,  ROW\_NUMBER()  OVER (ORDER BY P2.SalesOrderDetailID)  AS RowNum  

Predicate: No Predicate.

In [ ]:
-- Q11
USE AdventureWorks2017
GO

DECLARE @BufferDays FLOAT = 1.5;
DECLARE @PageSize INT = 20; -- Define the number of rows per page
DECLARE @PageNumber INT = 1; -- Define the page number you want to retrieve

-- CTE that returns the number of days to remanufacture products sold
WITH DaysManProd AS 
(
    SELECT
        OD.SalesOrderDetailID
        , OD.ProductID
        , P.DaysToManufacture
    FROM 
        Production.Product AS P
        INNER JOIN Sales.SalesOrderDetail AS OD
            ON P.ProductID = OD.ProductID
)

-- Add ROW_NUMBER() to the CTE
, PageCTE AS 
(
    SELECT
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
        , CEILING(P2.DaysToManufacture * @BufferDays) as RestockDate
        , ROW_NUMBER() OVER (ORDER BY P2.SalesOrderDetailID) AS RowNum
    FROM
        DaysManProd as P2
    GROUP BY
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
)

-- Use OFFSET and FETCH to segment the results
SELECT
    SalesOrderDetailID
    , ProductID
    , DaysToManufacture
    , RestockDate
FROM
    PageCTE
WHERE
    RowNum BETWEEN ((@PageNumber - 1) * @PageSize + 1) AND (@PageNumber * @PageSize);

-- FOR JSON AUTO, include_null_values;

# Corrected QUERY

## Medium

Q11  

Proposition: A query that returns the number of days to remanufacture products sold and its restock date.

Corrected: Removed buffered days and replaced (<span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">DaysToManufacture </span> <span style="font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 0);">*</span> <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"> @BufferDays) with</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">&nbsp;(</span><span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">DaysToManufacture * </span> <span style="color: rgb(33, 33, 33); font-family: Menlo, Monaco, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">OrderQty) for an accurate restock.</span>

In [ ]:
-- Q11
USE AdventureWorks2017
GO

DECLARE @PageSize INT = 20; -- Define the number of rows per page
DECLARE @PageNumber INT = 1; -- Define the page number you want to retrieve

-- CTE that returns the number of days to remanufacture products sold
WITH DaysManProd AS 
(
    SELECT
        OD.SalesOrderDetailID
        , OD.ProductID
        , P.DaysToManufacture
        , OD.OrderQty
    FROM 
        Production.Product AS P
        INNER JOIN Sales.SalesOrderDetail AS OD
            ON P.ProductID = OD.ProductID
)

-- Add ROW_NUMBER() to the CTE
, PageCTE AS 
(
    SELECT
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
        , CEILING(P2.DaysToManufacture * P2.OrderQty) as RestockDate
        , ROW_NUMBER() OVER (ORDER BY P2.SalesOrderDetailID) AS RowNum
    FROM
        DaysManProd as P2
    GROUP BY
        P2.SalesOrderDetailID
        , P2.ProductID
        , P2.DaysToManufacture
        , CEILING(P2.DaysToManufacture * P2.OrderQty)
)

-- Use OFFSET and FETCH to segment the results
SELECT
    SalesOrderDetailID
    , ProductID
    , DaysToManufacture
    , RestockDate
FROM
    PageCTE
WHERE
    RowNum BETWEEN ((@PageNumber - 1) * @PageSize + 1) AND (@PageNumber * @PageSize);

-- FOR JSON AUTO, include_null_values;

* * *

# Worst Query

## Complex

Q18

Proposition: A query that returns employees in a requested country. Return the employees' id, name, country, currency, year hired and age when hired.  

Table: dbo.DimEmployee  as e,  dbo.DimSalesTerritory as s and   dbo.DimCurrency  as c.  

Column: e.EmployeeKey, e.lastName, e.FirstName,  e.SalesTerritoryCountry, c.CurrencyName,  YEAR(e.HireDate)  AS  HireYear and  dbo.AgeWhenHired(e.BirthDate, e.HireDate)  AS  AgeWhenHired  

Predicate: No Predicate.

In [ ]:
-- Q18 
USE AdventureWorksDW2017
GO

-- Scalar Function returns Employee's age when they were hired
CREATE OR ALTER FUNCTION dbo.AgeWhenHired
(
    -- PARAMETERS
    @dateOfBirth DATE,
    @hireDate DATE
)
RETURNS INT AS
BEGIN
    --  Body
    DECLARE @result INT
    SELECT @result = DATEDIFF(YEAR, @dateOfBirth, @hireDate)
    RETURN @result
END
GO

-- Variable to input a sales country
DECLARE @salesCountry NVARCHAR(50) = 'CANADA'

-- Outer Query that returns info for employees in a requested country:
-- id, name, country, currency, year hired, and age when hired
SELECT DISTINCT
    e.EmployeeKey, e.lastName, e.FirstName,
    e.SalesTerritoryCountry, c.CurrencyName,
    YEAR(e.HireDate) AS HireYear,
    dbo.AgeWhenHired(e.BirthDate, e.HireDate) AS AgeWhenHired
FROM
(
    -- Inner Query that returns info for employees in a requested country
    SELECT 
        e.SalesTerritoryKey, e.EmployeeKey, e.lastName, e.FirstName,
        s.SalesTerritoryCountry, e.BirthDate, e.HireDate
    FROM 
        dbo.DimEmployee as e
        LEFT JOIN dbo.DimSalesTerritory as s
            ON e.SalesTerritoryKey = s.SalesTerritoryKey
    WHERE
        s.SalesTerritoryCountry = @salesCountry
) AS e LEFT JOIN dbo.FactInternetSales as f
        ON e.SalesTerritoryKey = f.SalesTerritoryKey
    LEFT JOIN dbo.DimCurrency as c
        ON f.CurrencyKey = c.CurrencyKey
GROUP BY
    e.EmployeeKey, e.lastName, e.FirstName,
    e.SalesTerritoryCountry, c.CurrencyName,
    e.HireDate, e.BirthDate
ORDER BY
    e.EmployeeKey, c.CurrencyName

-- FOR JSON AUTO, include_null_values;

* * *

# Corrected Query

## Complex

Q18

Proposition: A query that returns employees in a requested country and the currencies they handle. Return the employees' id, name, country, currency, year hired and age when hired.  

Corrected:

- Add aggregate function: COUNT(DISTINCT C.CurrencyKey) AS 'NumberOfCurriencies'
- Improved readability: 
    - Changed to PascalCase 
    - Variable names are self documenting: @result renamed to @AgeWhenHired  
    - Concated Employees' names AS FullName

In [ ]:
-- Q18 
USE AdventureWorksDW2017
GO

-- Scalar Function returns Employee's age when they were hired
CREATE OR ALTER FUNCTION dbo.AgeWhenHired
(
    -- PARAMETERS
    @DateOfBirth DATE,
    @HireDate DATE
)
RETURNS INT AS
BEGIN
    --  Body
    DECLARE @AgeWhenHired INT
    SELECT @AgeWhenHired = DATEDIFF(YEAR, @dateOfBirth, @hireDate)
    RETURN @AgeWhenHired
END
GO

-- Variable to input a sales country
DECLARE @SalesCountry NVARCHAR(50) = 'CANADA'

-- Outer Query that returns info for employees in a requested country:
-- id, name, country, number of currencies, year hired, and age when hired
SELECT DISTINCT
    E.EmployeeKey
    , CONCAT(E.LastName, ' ', E.FirstName) AS 'FullName'
    , E.SalesTerritoryCountry
    , COUNT(DISTINCT C.CurrencyKey) AS 'NumberOfCurriencies'
    , YEAR(E.HireDate) AS HireYear
    , dbo.AgeWhenHired(E.BirthDate, E.HireDate) AS AgeWhenHired
FROM
(
    -- Inner Query that returns info for employees in a requested country
    SELECT 
        E.SalesTerritoryKey
        , E.EmployeeKey
        , E.lastName
        , E.FirstName
        , S.SalesTerritoryCountry
        , E.BirthDate
        , E.HireDate
    FROM 
        dbo.DimEmployee as E
        LEFT JOIN dbo.DimSalesTerritory as S
            ON E.SalesTerritoryKey = S.SalesTerritoryKey
    WHERE
        S.SalesTerritoryCountry = @SalesCountry
) AS E LEFT JOIN dbo.FactInternetSales as F
        ON E.SalesTerritoryKey = F.SalesTerritoryKey
    LEFT JOIN dbo.DimCurrency as C
        ON F.CurrencyKey = C.CurrencyKey
GROUP BY
    E.EmployeeKey
    , CONCAT(E.LastName, ' ', E.FirstName)
    , E.SalesTerritoryCountry
    , YEAR(E.HireDate)
    , E.BirthDate
    , E.HireDate
ORDER BY
    E.EmployeeKey

-- FOR JSON AUTO, include_null_values;